In [ ]:
!pip3 install matplotlib python-bidi arabic_reshaper 

In [ ]:
# Author: Erfan Kheyrollahi <github.com/ekm507>
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import ImageFont
from matplotlib.font_manager import FontProperties
import os
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np

In [ ]:

def _get_persian_display_text(text, font_prop):
    """یک تابع کمکی برای آماده‌سازی متن فارسی برای نمایش در نمودار."""
    reshaped_text = arabic_reshaper.reshape(text)
    return get_display(reshaped_text)

def analyze_and_compare_fonts(font_paths, font_size=50, output_basename="font_comparison"):
    """
    پهنای حروف را برای لیستی از فونت‌ها محاسبه، نرمال‌سازی و مقایسه می‌کند.
    دو نمودار خروجی تولید می‌کند: یکی برای مقایسه منحنی‌ها و دیگری برای میانگin کل.

    Args:
        font_paths (list): لیستی از مسیرهای فایل‌های فونت (ttf. یا otf.).
        font_size (int): اندازه فونت برای محاسبه پهنا.
        output_basename (str): نام پایه برای فایل‌های خروجی نمودار.
    """
    if not font_paths:
        print("خطا: لیستی از فونت‌ها ارائه نشده است.")
        return

    all_normalized_widths = []
    font_data = {}

    # استفاده از فونت اول برای رندر کردن عناوین نمودار
    try:
        persian_font_prop = FontProperties(fname=font_paths[0])
    except Exception:
        print(f"هشدار: فونت '{font_paths[0]}' برای عناوین یافت نشد. از فونت پیش‌فرض استفاده می‌شود.")
        persian_font_prop = FontProperties()


    print("--- شروع فرآیند تحلیل فونت‌ها ---")
    for font_path in font_paths:
        try:
            font = ImageFont.truetype(font_path, font_size)
            font_name = os.path.basename(font_path).replace('.ttf', '').replace('.otf', '')
            print(f"\nدر حال پردازش فونت: {font_name}")
        except IOError:
            print(f"خطا: فایل فونت '{font_path}' یافت نشد یا معتبر نیست. از این فونت صرف‌نظر می‌شود.")
            continue

        ZWJ = '\u200d'
        persian_alphabet = "اآبپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی"
        current_font_widths = []

        for char in persian_alphabet:
            current_font_widths.append(font.getlength(char))
            current_font_widths.append(font.getlength(char + ZWJ))
            current_font_widths.append(font.getlength(ZWJ + char + ZWJ))
            current_font_widths.append(font.getlength(ZWJ + char))

        if not current_font_widths:
            continue

        # نرمال‌سازی پهناها
        average_width = np.mean(current_font_widths)
        normalized_widths = [w / average_width for w in current_font_widths]

        font_data[font_name] = normalized_widths
        all_normalized_widths.extend(normalized_widths)

        print(f"میانگین پهنای اصلی: {average_width:.2f} پیکسل. پردازش کامل شد.")

    if not font_data:
        print("هیچ فونتی با موفقیت پردازش نشد. برنامه خاتمه می‌یابد.")
        return

    # --- ترسیم نمودار اول: مقایسه منحنی‌های توزیع ---
    output_overlay_path = f"{output_basename}_overlay.png"
    print(f"\nدر حال ترسیم نمودار مقایسه و ذخیره در '{output_overlay_path}'...")

    sns.set_theme(style="whitegrid", palette="muted")
    plt.figure(figsize=(14, 8))

    ax1 = plt.gca()
    for font_name, widths in font_data.items():
        sns.kdeplot(widths, ax=ax1, label=font_name, linewidth=2.5)

    ax1.legend(prop=persian_font_prop, fontsize=12)
    title1 = _get_persian_display_text("مقایسه توزیع پهنای نرمال‌شده فونت‌ها", persian_font_prop)
    xlabel1 = _get_persian_display_text("پهنای نرمال‌شده (میانگین = ۱)", persian_font_prop)
    ylabel1 = _get_persian_display_text("چگالی (Density)", persian_font_prop)

    ax1.set_title(title1, fontproperties=persian_font_prop, fontsize=20, pad=20)
    ax1.set_xlabel(xlabel1, fontproperties=persian_font_prop, fontsize=14)
    ax1.set_ylabel(ylabel1, fontproperties=persian_font_prop, fontsize=14)

    plt.savefig(output_overlay_path, dpi=150, bbox_inches='tight')
    plt.close()

    # --- ترسیم نمودار دوم: میانگین کل توزیع‌ها ---
    output_average_path = f"{output_basename}_average.png"
    print(f"در حال ترسیم نمودار میانگین و ذخیره در '{output_average_path}'...")

    plt.figure(figsize=(12, 7))
    ax2 = sns.histplot(all_normalized_widths, kde=True, bins=30, line_kws={'linewidth': 3}, color='teal')

    title2 = _get_persian_display_text("توزیع میانگین پهنای نرمال‌شده (تمام فونت‌ها)", persian_font_prop)

    ax2.set_title(title2, fontproperties=persian_font_prop, fontsize=18, pad=20)
    ax2.set_xlabel(xlabel1, fontproperties=persian_font_prop, fontsize=14) # Using same xlabel
    ax2.set_ylabel(_get_persian_display_text("تعداد (فراوانی)", persian_font_prop), fontproperties=persian_font_prop, fontsize=14)

    plt.savefig(output_average_path, dpi=150, bbox_inches='tight')
    plt.close()

    print("\n✅ تحلیل و ترسیم نمودارها با موفقیت انجام شد.")

In [ ]:
list_of_fonts_to_analyze = [
    "fonts/Nahid.ttf",
    'fonts/sahel-font-master/dist/Sahel.ttf',
    'fonts/XB Kayhan.ttf',
    'fonts/Vazirmatn-Regular.ttf',
    'fonts/XP Ziba.ttf',
]

In [ ]:
# --- راهنمای استفاده ---
# ۱. مطمئن شوید کتابخانه‌های matplotlib, seaborn, numpy, arabic_reshaper, python-bidi نصب هستند.
# ۲. لیستی از مسیرهای فونت‌های خود را در متغیر list_of_fonts قرار دهید.
#    فایل‌های فونت باید در کنار اسکریپت باشند یا مسیر کامل آن‌ها را وارد کنید.

In [ ]:
# بررسی وجود حداقل یک فونت
if not list_of_fonts_to_analyze or not os.path.exists(list_of_fonts_to_analyze[0]):
    print("هشدار: لیست فونت‌ها خالی است یا فونت Vazirmatn-Regular.ttf یافت نشد.")
    print("برنامه یک فونت نمونه ایجاد می‌کند تا بتوانید آن را تست کنید.")
    try:
        from PIL import Image
        img = Image.new('RGB', (1, 1))
        # تلاش برای استفاده از یک فونت پیش‌فرض سیستمی در صورت عدم وجود فونت وزیر
        try:
            font = ImageFont.truetype("tahoma.ttf", 40)
        except IOError:
            font = ImageFont.load_default()
        font.save("sample_font.otf")
        list_of_fonts_to_analyze.append("sample_font.otf")
        print("فونت نمونه 'sample_font.otf' ایجاد شد.")
    except Exception as e:
        print(f"خطا در ایجاد فونت نمونه: {e}")analyze_and_compare_fonts(list_of_fonts_to_analyze, font_size=40)


In [ ]:
analyze_and_compare_fonts(list_of_fonts_to_analyze, font_size=40)